In [1]:
import nltk
import string
import PyPDF2
import os
import spacy
import string
import numpy as np

nlp = spacy.load('en_core_web_sm')

import matplotlib.pyplot as plt
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.models import Phrases
from gensim.models.phrases import Phraser

from wordcloud import WordCloud
import pandas as pd


### Uploading Reports

In [2]:
#Loop that goes through each of the reports in the 'reports' folder, and turns them into strings, and removes the line breaks.

pdf_folder = 'reports'
pdf_texts = {}

# Loop through all PDFs
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        report_name = filename.replace('.pdf', '').lower()
        with open(os.path.join(pdf_folder, filename), 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                content = page.extract_text()
                if content:
                    text += content + "\n"
            pdf_texts[report_name] = text

In [7]:
ubs24 = pdf_texts['ubs_24']
ubs23 = pdf_texts['ubs_23']
ubs22 = pdf_texts['ubs_22']
ubs21 = pdf_texts['ubs_21']
pwc24 = pdf_texts['pwc_24']
pwc23 = pdf_texts['pwc_23']
# There was no PWC 2021 report, but i chose to continue without it
citi24 = pdf_texts['citi_24']
citi23 = pdf_texts['citi_23']
citi22 = pdf_texts['citi_22']
citi21 = pdf_texts['citi_21']
campden24 = pdf_texts['campden_24']
campden23 = pdf_texts['campden_23']
campden22 = pdf_texts['campden_22']
campden21 = pdf_texts['campden_21']

KeyError: 'pwc_22.pdf'